# Pipeline Parity Audit
**Key finding**: Official baseline (h=64 3L GRU, 68K params) scores 0.2761 LB with val=0.2595.
Our baseline_match (same h=64 3L but with input_proj+LayerNorm+MLP+derived+zscore): val=0.2738, LB=0.2394.

**Same architecture, 0.037 LB gap.** Official has +0.017 POSITIVE gap. Ours has -0.034 NEGATIVE gap.

Two configs to isolate the cause:
- **v1 (full parity)**: vanilla+linear, raw32, no norm, MSE loss, dropout=0 → match official baseline exactly
- **v2 (architecture only)**: vanilla+linear, BUT with our derived42 + z-score + combined loss → isolate architecture effect

This is NOT a val score test. We're looking at the val-LB GAP direction.
If parity models get positive or near-zero gap, we've found the generalization path.

In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup — clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train v1 (FULL PARITY — raw32, no norm, MSE, vanilla+linear)
import os, subprocess, sys
os.chdir("/content/competition_package")

CONFIG = "configs/gru_parity_v1.yaml"
SEEDS = [42, 43, 44]

print(f"=== PARITY v1: Full baseline match ===")
print(f"Config: {CONFIG}")
print(f"vanilla+linear, raw32, no norm, MSE loss, dropout=0")
print("=" * 60, flush=True)

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"Training gru_parity_v1 seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    proc.wait()
    if proc.returncode != 0:
        print(f"ERROR: seed {seed} failed with return code {proc.returncode}")

print(f"\nv1 done!")

In [ ]:
# Cell 3: Train v2 (ARCHITECTURE ONLY — vanilla+linear but with our recipe)
import os, subprocess, sys
os.chdir("/content/competition_package")

CONFIG = "configs/gru_parity_v2.yaml"
SEEDS = [42, 43, 44]

print(f"=== PARITY v2: Vanilla arch + our recipe ===")
print(f"Config: {CONFIG}")
print(f"vanilla+linear, derived42, z-score, combined loss")
print("=" * 60, flush=True)

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"Training gru_parity_v2 seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    proc.wait()
    if proc.returncode != 0:
        print(f"ERROR: seed {seed} failed with return code {proc.returncode}")

print(f"\nv2 done!")

In [ ]:
# Cell 4: Evaluation — compare both configs
import os, glob, torch
os.chdir("/content/competition_package")

# Reference points
OFFICIAL_BASELINE_VAL = 0.2595
OFFICIAL_BASELINE_LB = 0.2761
OUR_BM_VAL = 0.2738  # baseline_match with our full pipeline
OUR_BM_LB = 0.2394
TIGHTWD_VAL = 0.2660  # tightwd_v2 mean val

for config_name in ["gru_parity_v1", "gru_parity_v2"]:
    print(f"\n{'='*70}")
    print(f"  {config_name}")
    print(f"{'='*70}")
    
    results = []
    for pt in sorted(glob.glob(f"logs/{config_name}_seed*.pt")):
        basename = os.path.basename(pt)
        if '_epoch' in basename:
            continue
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
        score = float(ckpt.get("best_score", 0))
        epoch = ckpt.get("best_epoch", "N/A")
        results.append((basename, score, epoch))
    
    if not results:
        print("  No checkpoints found!")
        continue
    
    print(f"{'Model':<50} {'Val Score':>10} {'Epoch':>6}")
    print("-" * 70)
    for name, score, epoch in results:
        print(f"{name:<50} {score:>10.4f} {str(epoch):>6}")
    
    scores = [s for _, s, _ in results]
    mean_val = sum(scores) / len(scores)
    
    print(f"\nMean val: {mean_val:.4f}")
    print(f"vs official baseline val: {mean_val - OFFICIAL_BASELINE_VAL:+.4f}")
    print(f"vs our baseline_match val: {mean_val - OUR_BM_VAL:+.4f}")
    print(f"vs tightwd_v2 val: {mean_val - TIGHTWD_VAL:+.4f}")

print(f"\n{'='*70}")
print("INTERPRETATION GUIDE:")
print(f"Official baseline: val={OFFICIAL_BASELINE_VAL}, LB={OFFICIAL_BASELINE_LB} (gap +0.017)")
print(f"Our baseline_match: val={OUR_BM_VAL}, LB={OUR_BM_LB} (gap -0.034)")
print(f"Our tightwd_v2: val={TIGHTWD_VAL}, LB=0.258 (gap -0.008)")
print()
print("If parity_v1 val near 0.26 → matches official baseline, submit for gap test")
print("If parity_v2 val > tightwd_v2 → input_proj+MLP was HURTING generalization")
print("If parity_v1 val << 0.25 → our training procedure fundamentally differs")

In [ ]:
# Cell 5: Strip checkpoints + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs("logs/slim", exist_ok=True)

for pattern in ["gru_parity_v1", "gru_parity_v2"]:
    for pt in sorted(glob.glob(f"logs/{pattern}*.pt")):
        basename = os.path.basename(pt)
        if '_epoch' in basename:
            continue
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
        slim = {
            "model_state_dict": ckpt["model_state_dict"],
            "config": ckpt.get("config", {}),
            "best_score": ckpt.get("best_score", None),
            "best_epoch": ckpt.get("best_epoch", None),
        }
        out = f"logs/slim/{basename}"
        torch.save(slim, out)
        orig = os.path.getsize(pt) / 1e6
        new = os.path.getsize(out) / 1e6
        print(f"{basename}: {orig:.1f}MB -> {new:.1f}MB")

for npz in sorted(glob.glob("logs/normalizer_gru_parity*.npz")):
    basename = os.path.basename(npz)
    shutil.copy(npz, f"logs/slim/{basename}")
    print(f"Copied {basename}")

shutil.make_archive("/content/parity_audit", "zip",
                     "/content/competition_package/logs/slim")
sz = os.path.getsize("/content/parity_audit.zip") / 1e6
print(f"\nparity_audit.zip: {sz:.1f}MB")

shutil.copy("/content/parity_audit.zip",
            "/content/drive/MyDrive/wunderfund/parity_audit.zip")
print("Saved to Drive: MyDrive/wunderfund/parity_audit.zip")